<a href="https://colab.research.google.com/github/Bhargavbollu/Bhargavbollu/blob/main/Module_01_Lab_02_MLPractice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Machine learning terms and metrics

FMML Module 1, Lab 2<br>


 In this lab, we will show a part of the ML pipeline by extracting features, training and testing

In [1]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn import datasets
# set randomseed
rng = np.random.default_rng(seed=42)

In this lab, we will use the California Housing dataset. There are 20640 samples, each with 8 attributes like income of the block, age of the houses per district etc. The task is to predict the cost of the houses per district.

Let us download and examine the dataset.

In [2]:
 dataset =  datasets.fetch_california_housing()
 # print(dataset.DESCR)  # uncomment this if you want to know more about this dataset
 # print(dataset.keys())  # if you want to know what else is there in this dataset
 dataset.target = dataset.target.astype(np.int) # so that we can classify
 print(dataset.data.shape)
 print(dataset.target.shape)

(20640, 8)
(20640,)


<ipython-input-2-60ae2e9a125e>:4: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dataset.target = dataset.target.astype(np.int) # so that we can classify


Here is a function for calculating the 1-nearest neighbours

In [3]:
def NN1(traindata, trainlabel, query):
  diff  = traindata - query  # find the difference between features. Numpy automatically takes care of the size here
  sq = diff*diff # square the differences
  dist = sq.sum(1) # add up the squares
  label = trainlabel[np.argmin(dist)] # our predicted label is the label of the training data which has the least distance from the query
  return label

def NN(traindata, trainlabel, testdata):
  # we will run nearest neighbour for each sample in the test data
  # and collect the predicted classes in an array using list comprehension
  predlabel = np.array([NN1(traindata, trainlabel, i) for i in testdata])
  return predlabel

We will also define a 'random classifier', which randomly allots labels to each sample

In [4]:
def RandomClassifier(traindata, trainlabel, testdata):
  # in reality, we don't need these arguments

  classes = np.unique(trainlabel)
  rints = rng.integers(low=0, high=len(classes), size=len(testdata))
  predlabel = classes[rints]
  return predlabel

Let us define a metric 'Accuracy' to see how good our learning algorithm is. Accuracy is the ratio of the number of correctly classified samples to the total number of samples. The higher the accuracy, the better the algorithm.

In [5]:
def Accuracy(gtlabel, predlabel):
  assert len(gtlabel)==len(predlabel), "Length of the groundtruth labels and predicted labels should be the same"
  correct = (gtlabel==predlabel).sum() # count the number of times the groundtruth label is equal to the predicted label.
  return correct/len(gtlabel)

Let us make a function to split the dataset with the desired probability.

In [6]:
def split(data, label, percent):
  # generate a random number for each sample
  rnd = rng.random(len(label))
  split1 = rnd<percent
  split2 = rnd>=percent
  split1data = data[split1,:]
  split1label = label[split1]
  split2data = data[split2,:]
  split2label = label[split2]
  return split1data, split1label, split2data, split2label

We will reserve 20% of our dataset as the test set. We will not change this portion throughout our experiments

In [7]:
testdata, testlabel, alltraindata, alltrainlabel = split(dataset.data, dataset.target, 20/100)
print('Number of test samples = ', len(testlabel))
print('Number of other samples = ', len(alltrainlabel))
print('Percent of test data = ', len(testlabel)*100/len(dataset.target),'%')

Number of test samples =  4144
Number of other samples =  16496
Percent of test data =  20.07751937984496 %


## Experiments with splits

Let us reserve some of our train data as a validation set

In [8]:
traindata, trainlabel, valdata, vallabel = split(alltraindata, alltrainlabel, 75/100)

What is the accuracy of our classifiers on the train dataset?

In [9]:
trainpred = NN(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Train accuracy using nearest neighbour is ", trainAccuracy)

trainpred = RandomClassifier(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Train accuracy using random classifier is ", trainAccuracy)

Train accuracy using nearest neighbour is  1.0
Train accuracy using random classifier is  0.164375808538163


For nearest neighbour, the train accuracy is always 1. The accuracy of the random classifier is close to 1/(number of classes) which is 0.1666 in our case.

Let us predict the labels for our validation set and get the accuracy

In [10]:
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using nearest neighbour is ", valAccuracy)

valpred = RandomClassifier(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using random classifier is ", valAccuracy)

Validation accuracy using nearest neighbour is  0.34108527131782945
Validation accuracy using random classifier is  0.1688468992248062


Validation accuracy of nearest neighbour is considerably less than its train accuracy while the validation accuracy of random classifier is the same. However, the validation accuracy of nearest neighbour is twice that of the random classifier.

Now let us try another random split and check the validation accuracy

In [11]:
traindata, trainlabel, valdata, vallabel = split(alltraindata, alltrainlabel, 75/100)
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy of nearest neighbour is ", valAccuracy)

Validation accuracy of nearest neighbour is  0.34048257372654156


You can run the above cell multiple times to try with different random splits.
We notice that the accuracy is different for each run, but close together.

Now let us compare it with the accuracy we get on the test dataset.

In [12]:
testpred = NN(alltraindata, alltrainlabel, testdata)
testAccuracy = Accuracy(testlabel, testpred)
print('Test accuracy is ', testAccuracy)

Test accuracy is  0.34917953667953666


### Try it out for yourself and answer:
1. How is the accuracy of the validation set affected if we increase the percentage of validation set? What happens when we reduce it?
2. How does the size of the train and validation set affect how well we can predict the accuracy on the test set using the validation set?
3. What do you think is a good percentage to reserve for the validation set so that thest two factors are balanced?

Answer for both nearest neighbour and random classifier. You can note down the values for your experiments and plot a graph using  <a href=https://matplotlib.org/stable/gallery/lines_bars_and_markers/step_demo.html#sphx-glr-gallery-lines-bars-and-markers-step-demo-py>plt.plot<href>. Check also for extreme values for splits, like 99.9% or 0.1%

## Multiple Splits

One way to get more accurate estimates for the test accuracy is by using <b>crossvalidation</b>. Here, we will try a simple version, where we do multiple train/val splits and take the average of validation accuracies as the test accuracy estimation. Here is a function for doing this. Note that this function will take a long time to execute.

In [13]:
# you can use this function for random classifier also
def AverageAccuracy(alldata, alllabel, splitpercent, iterations, classifier=NN):
  accuracy = 0
  for ii in range(iterations):
    traindata, trainlabel, valdata, vallabel = split(alldata, alllabel, splitpercent)
    valpred = classifier(traindata, trainlabel, valdata)
    accuracy += Accuracy(vallabel, valpred)
  return accuracy/iterations # average of all accuracies

In [14]:
print('Average validation accuracy is ', AverageAccuracy(alltraindata, alltrainlabel, 75/100, 10, classifier=NN))
testpred = NN(alltraindata, alltrainlabel, testdata)
print('test accuracy is ',Accuracy(testlabel, testpred) )

Average validation accuracy is  0.33584635395170215
test accuracy is  0.34917953667953666


This is a very simple way of doing cross-validation. There are many well-known algorithms for cross-validation, like k-fold cross-validation, leave-one-out etc. This will be covered in detail in a later module. For more information about cross-validation, check <a href=https://en.wikipedia.org/wiki/Cross-validation_(statistics)>Cross-validatioin (Wikipedia)</a>

### Questions
1. Does averaging the validation accuracy across multiple splits give more consistent results?
2. Does it give more accurate estimate of test accuracy?
3. What is the effect of the number of iterations on the estimate? Do we get a better estimate with higher iterations?
4. Consider the results you got for the previous questions. Can we deal with a very small train dataset or validation dataset by increasing the iterations?


In [ ]:
cells": [
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "view-in-github",
        "colab_type": "text"
      },
      "source": [
        "<a href=\"https://colab.research.google.com/github/cygnuxxs/FMML_Assignments/blob/main/Module_01_Lab_02_MLPractice.ipynb\" target=\"_parent\"><img src=\"https://colab.research.google.com/assets/colab-badge.svg\" alt=\"Open In Colab\"/></a>"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "3Eu9VZbF01eq"
      },
      "source": [
        "# Machine learning terms and metrics\n",
        "\n",
        "FMML Module 1, Lab 2<br>\n",
        "\n",
        "\n",
        " In this lab, we will show a part of the ML pipeline by extracting features, training and testing"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": 2,
      "metadata": {
        "id": "8qBvyEem0vLi"
      },
      "outputs": [],
      "source": [
        "import matplotlib.pyplot as plt\n",
        "import numpy as np\n",
        "from sklearn import datasets\n",
        "# set randomseed\n",
        "rng = np.random.default_rng(seed=42)"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "u3t59g5s1HfC"
      },
      "source": [
        "In this lab, we will use the California Housing dataset. There are 20640 samples, each with 8 attributes like income of the block, age of the houses per district etc. The task is to predict the cost of the houses per district.\n",
        "\n",
        "Let us download and examine the dataset."
      ]
    },
    {
      "cell_type": "code",
      "execution_count": 3,
      "metadata": {
        "colab": {
          "base_uri": "https://localhost:8080/"
        },
        "id": "8LpqjN991GGJ",
        "outputId": "12e7da63-4137-4f7f-813a-9d8b7b1e7ee8"
      },
      "outputs": [
        {
          "output_type": "stream",
          "name": "stdout",
          "text": [
            "(20640, 8)\n",
            "(20640,)\n"
          ]
        },
        {
          "output_type": "stream",
          "name": "stderr",
          "text": [
            "<ipython-input-3-60ae2e9a125e>:4: DeprecationWarning: np.int is a deprecated alias for the builtin int. To silence this warning, use int by itself. Doing this will not modify any behavior and is safe. When replacing np.int, you may wish to use e.g. np.int64 or np.int32 to specify the precision. If you wish to review your current use, check the release note link for additional information.\n",
            "Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations\n",
            "  dataset.target = dataset.target.astype(np.int) # so that we can classify\n"
          ]
        }
      ],
      "source": [
        " dataset =  datasets.fetch_california_housing()\n",
        " # print(dataset.DESCR)  # uncomment this if you want to know more about this dataset\n",
        " # print(dataset.keys())  # if you want to know what else is there in this dataset\n",
        " dataset.target = dataset.target.astype(np.int) # so that we can classify\n",
        " print(dataset.data.shape)\n",
        " print(dataset.target.shape)"
      ]
    },
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "iNx4174W5xRg"
      },
      "source": [
        "Here is a function for calculating the 1-nearest neighbours"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": 4,
      "metadata": {
        "id": "07zpydQj1hIQ"
      },
      "outputs": [],
      "source": [
        "def NN1(traindata, trainlabel, query):\n",
        "  diff  = traindata - query  # find the difference between features. Numpy automatically takes care of the size here\n",
        "  sq = diff*diff # square the differences\n",
        "  dist = sq.sum(1) # add up the squares\n",
        "  label = trainlabel[np.argmin(dist)] # our predicted label is the label of the training data which has the least distance from the query\n",
        "  return label\n",
        "\n",
        "def NN(traindata, trainlabel, testdata):\n",
        "  # we will run nearest neighbour for each sample in the test data\n",
        "  # and collect the predicted classes in an array using list comprehension\n",
        "  predlabel = np.array([NN1(traindata, trainlabel, i) for i in testdata])\n",
        "  return predlabel"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "03JktkfIGaje"
      },
      "source": [
        "We will also define a 'random classifier', which randomly allots labels to each sample"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": 5,
      "metadata": {
        "id": "fogWAtjyGhAH"
      },
      "outputs": [],
      "source": [
        "def RandomClassifier(traindata, trainlabel, testdata):\n",
        "  # in reality, we don't need these arguments\n",
        "\n",
        "  classes = np.unique(trainlabel)\n",
        "  rints = rng.integers(low=0, high=len(classes), size=len(testdata))\n",
        "  predlabel = classes[rints]\n",
        "  return predlabel"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "1Hjf1KHs7fU5"
      },
      "source": [
        "Let us define a metric 'Accuracy' to see how good our learning algorithm is. Accuracy is the ratio of the number of correctly classified samples to the total number of samples. The higher the accuracy, the better the algorithm."
      ]
    },
    {
      "cell_type": "code",
      "execution_count": 6,
      "metadata": {
        "id": "ouuCqWU07bz-"
      },
      "outputs": [],
      "source": [
        "def Accuracy(gtlabel, predlabel):\n",
        "  assert len(gtlabel)==len(predlabel), \"Length of the groundtruth labels and predicted labels should be the same\"\n",
        "  correct = (gtlabel==predlabel).sum() # count the number of times the groundtruth label is equal to the predicted label.\n",
        "  return correct/len(gtlabel)"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "4vJFwBFa9Klw"
      },
      "source": [
        "Let us make a function to split the dataset with the desired probability."
      ]
    },
    {
      "cell_type": "code",
      "execution_count": 7,
      "metadata": {
        "id": "Ko0VzpSM2Tdi"
      },
      "outputs": [],
      "source": [
        "def split(data, label, percent):\n",
        "  # generate a random number for each sample\n",
        "  rnd = rng.random(len(label))\n",
        "  split1 = rnd<percent\n",
        "  split2 = rnd>=percent\n",
        "  split1data = data[split1,:]\n",
        "  split1label = label[split1]\n",
        "  split2data = data[split2,:]\n",
        "  split2label = label[split2]\n",
        "  return split1data, split1label, split2data, split2label"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "AcK3LEAJ_LGC"
      },
      "source": [
        "We will reserve 20% of our dataset as the test set. We will not change this portion throughout our experiments"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": 8,
      "metadata": {
        "colab": {
          "base_uri": "https://localhost:8080/"
        },
        "id": "bBZkHBLJ1iU-",
        "outputId": "93fa6e09-292e-4825-eb4a-6acc9d7565c2"
      },
      "outputs": [
        {
          "output_type": "stream",
          "name": "stdout",
          "text": [
            "Number of test samples =  4144\n",
            "Number of other samples =  16496\n",
            "Percent of test data =  20.07751937984496 %\n"
          ]
        }
      ],
      "source": [
        "testdata, testlabel, alltraindata, alltrainlabel = split(dataset.data, dataset.target, 20/100)\n",
        "print('Number of test samples = ', len(testlabel))\n",
        "print('Number of other samples = ', len(alltrainlabel))\n",
        "print('Percent of test data = ', len(testlabel)*100/len(dataset.target),'%')"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "a6Ss0Z6IAGNV"
      },
      "source": [
        "## Experiments with splits\n",
        "\n",
        "Let us reserve some of our train data as a validation set"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": 9,
      "metadata": {
        "id": "WFew2iry_7W7"
      },
      "outputs": [],
      "source": [
        "traindata, trainlabel, valdata, vallabel = split(alltraindata, alltrainlabel, 75/100)"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "60hiu4clFN1i"
      },
      "source": [
        "What is the accuracy of our classifiers on the train dataset?"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": 10,
      "metadata": {
        "colab": {
          "base_uri": "https://localhost:8080/"
        },
        "id": "DBlZDTHUFTZx",
        "outputId": "cadf743a-ae6b-4e97-bf5f-708b1d3cb57e"
      },
      "outputs": [
        {
          "output_type": "stream",
          "name": "stdout",
          "text": [
            "Train accuracy using nearest neighbour is  1.0\n",
            "Train accuracy using random classifier is  0.164375808538163\n"
          ]
        }
      ],
      "source": [
        "trainpred = NN(traindata, trainlabel, traindata)\n",
        "trainAccuracy = Accuracy(trainlabel, trainpred)\n",
        "print(\"Train accuracy using nearest neighbour is \", trainAccuracy)\n",
        "\n",
        "trainpred = RandomClassifier(traindata, trainlabel, traindata)\n",
        "trainAccuracy = Accuracy(trainlabel, trainpred)\n",
        "print(\"Train accuracy using random classifier is \", trainAccuracy)"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "7h08-9gJDtSy"
      },
      "source": [
        "For nearest neighbour, the train accuracy is always 1. The accuracy of the random classifier is close to 1/(number of classes) which is 0.1666 in our case.\n",
        "\n",
        "Let us predict the labels for our validation set and get the accuracy"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": 11,
      "metadata": {
        "colab": {
          "base_uri": "https://localhost:8080/"
        },
        "id": "4h7bXoW_2H3v",
        "outputId": "01d04389-5114-4894-c3eb-51a65d5f0a73"
      },
      "outputs": [
        {
          "output_type": "stream",
          "name": "stdout",
          "text": [
            "Validation accuracy using nearest neighbour is  0.34108527131782945\n",
            "Validation accuracy using random classifier is  0.1688468992248062\n"
          ]
        }
      ],
      "source": [
        "valpred = NN(traindata, trainlabel, valdata)\n",
        "valAccuracy = Accuracy(vallabel, valpred)\n",
        "print(\"Validation accuracy using nearest neighbour is \", valAccuracy)\n",
        "\n",
        "valpred = RandomClassifier(traindata, trainlabel, valdata)\n",
        "valAccuracy = Accuracy(vallabel, valpred)\n",
        "print(\"Validation accuracy using random classifier is \", valAccuracy)"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "py9bLguFEjfg"
      },
      "source": [
        "Validation accuracy of nearest neighbour is considerably less than its train accuracy while the validation accuracy of random classifier is the same. However, the validation accuracy of nearest neighbour is twice that of the random classifier.\n",
        "\n",
        "Now let us try another random split and check the validation accuracy"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": 12,
      "metadata": {
        "colab": {
          "base_uri": "https://localhost:8080/"
        },
        "id": "ujm3cyYzEntE",
        "outputId": "15116194-aed1-48a6-b7a3-75eb93221e93"
      },
      "outputs": [
        {
          "output_type": "stream",
          "name": "stdout",
          "text": [
            "Validation accuracy of nearest neighbour is  0.34048257372654156\n"
          ]
        }
      ],
      "source": [
        "traindata, trainlabel, valdata, vallabel = split(alltraindata, alltrainlabel, 75/100)\n",
        "valpred = NN(traindata, trainlabel, valdata)\n",
        "valAccuracy = Accuracy(vallabel, valpred)\n",
        "print(\"Validation accuracy of nearest neighbour is \", valAccuracy)"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "oSOx7U83EKie"
      },
      "source": [
        "You can run the above cell multiple times to try with different random splits.\n",
        "We notice that the accuracy is different for each run, but close together.\n",
        "\n",
        "Now let us compare it with the accuracy we get on the test dataset."
      ]
    },
    {
      "cell_type": "code",
      "execution_count": 13,
      "metadata": {
        "colab": {
          "base_uri": "https://localhost:8080/"
        },
        "id": "PNEZ5ToYBEDW",
        "outputId": "fce60e88-db53-4fd7-b43e-9b02119cadda"
      },
      "outputs": [
        {
          "output_type": "stream",
          "name": "stdout",
          "text": [
            "Test accuracy is  0.34917953667953666\n"
          ]
        }
      ],
      "source": [
        "testpred = NN(alltraindata, alltrainlabel, testdata)\n",
        "testAccuracy = Accuracy(testlabel, testpred)\n",
        "print('Test accuracy is ', testAccuracy)"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "w3dGD531K3gH"
      },
      "source": [
        "### Try it out for yourself and answer:\n",
        "1. How is the accuracy of the validation set affected if we increase the percentage of validation set? What happens when we reduce it?\n",
        "2. How does the size of the train and validation set affect how well we can predict the accuracy on the test set using the validation set?\n",
        "3. What do you think is a good percentage to reserve for the validation set so that these two factors are balanced?\n",
        "\n",
        "Answer for both nearest neighbour and random classifier. You can note down the values for your experiments and plot a graph using  <a href=https://matplotlib.org/stable/gallery/lines_bars_and_markers/step_demo.html#sphx-glr-gallery-lines-bars-and-markers-step-demo-py>plt.plot<href>. Check also for extreme values for splits, like 99.9% or 0.1%"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": 14,
      "metadata": {
        "colab": {
          "base_uri": "https://localhost:8080/"
        },
        "id": "T-Sgca9w9RDc",
        "outputId": "d5ceec62-27b7-438b-dce1-4ba954ca9def"
      },
      "outputs": [
        {
          "output_type": "stream",
          "name": "stdout",
          "text": [
            "Validation (75%) accuracy of nearest neighbour is  0.3504424778761062\n",
            "Validation (75%) accuracy using random classifier is  0.16283185840707964\n",
            "\n",
            "Validation (10%) accuracy of nearest neighbour is  0.29866630742287487\n",
            "Validation (10%) accuracy using random classifier is  0.1669136467735417\n"
          ]
        }
      ],
      "source": [
        "# Q1. Increasing the percentage of Validation Set\n",
        "traindata, trainlabel, valdata, vallabel = split(alltraindata, alltrainlabel, 90/100) # Increased from 75 to 90\n",
        "valpred = NN(traindata, trainlabel, valdata)\n",
        "valAccuracy = Accuracy(vallabel, valpred)\n",
        "print(\"Validation (75%) accuracy of nearest neighbour is \", valAccuracy)\n",
        "\n",
        "valpred = RandomClassifier(traindata, trainlabel, valdata)\n",
        "valAccuracy = Accuracy(vallabel, valpred)\n",
        "print(\"Validation (75%) accuracy using random classifier is \", valAccuracy)\n",
        "\n",
        "print()\n",
        "# Decreasing the percentage of Validation Set\n",
        "\n",
        "traindata, trainlabel, valdata, vallabel = split(alltraindata, alltrainlabel, 10/100) # Decreased from 75 to 10\n",
        "valpred = NN(traindata, trainlabel, valdata)\n",
        "valAccuracy = Accuracy(vallabel, valpred)\n",
        "print(\"Validation (10%) accuracy of nearest neighbour is \", valAccuracy)\n",
        "\n",
        "valpred = RandomClassifier(traindata, trainlabel, valdata)\n",
        "valAccuracy = Accuracy(vallabel, valpred)\n",
        "print(\"Validation (10%) accuracy using random classifier is \", valAccuracy)\n",
        "\n",
        "\n"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": 15,
      "metadata": {
        "colab": {
          "base_uri": "https://localhost:8080/"
        },
        "id": "wSKJRwi1B4Qw",
        "outputId": "bdb37f2d-4094-48e8-aa3c-81dc4a6bd95b"
      },
      "outputs": [
        {
          "output_type": "stream",
          "name": "stdout",
          "text": [
            "Percent of Train Data in the dataset: 50.24\n",
            "Percent of Test Data in the dataset: 24.49\n",
            "Percent of Validation Data in the dataset: 25.27\n",
            "Accuracy (Nearest Neighbour) of Test Set using Validation Set: 0.3079578139980825\n",
            "Accuracy (Random Classifier) of Test Set using Validation Set: 0.1695110258868648\n"
          ]
        }
      ],
      "source": [
        "# Q2. Let us take 50% of the dataset as the Train set\n",
        "# 25% of the dataset as the validation set\n",
        "# 25% of the dataset as the test set\n",
        "traindata, trainlabel, splitdata, splitlabel = split(dataset.data, dataset.target, 50/100)\n",
        "valdata, vallabel, testdata, testlabel = split(splitdata, splitlabel, 50/100)\n",
        "\n",
        "print(f\"Percent of Train Data in the dataset: {round(len(trainlabel)*100/len(dataset.target), 2)}\")\n",
        "print(f\"Percent of Test Data in the dataset: {round(len(testlabel)*100/len(dataset.target), 2)}\")\n",
        "print(f\"Percent of Validation Data in the dataset: {round(len(vallabel)*100/len(dataset.target), 2)}\")\n",
        "\n",
        "testpred = NN(testdata, testlabel, valdata)\n",
        "testpred2 = RandomClassifier(testdata, testlabel, valdata)\n",
        "\n",
        "accu = Accuracy(vallabel, testpred)\n",
        "accu2 = Accuracy(vallabel, testpred2)\n",
        "print(f\"Accuracy (Nearest Neighbour) of Test Set using Validation Set: {accu}\")\n",
        "print(f\"Accuracy (Random Classifier) of Test Set using Validation Set: {accu2}\")\n",
        "\n"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": 16,
      "metadata": {
        "colab": {
          "base_uri": "https://localhost:8080/"
        },
        "id": "Dt6KDLmLB57Y",
        "outputId": "b58b019b-b6c2-49df-e209-16b955f244f1"
      },
      "outputs": [
        {
          "output_type": "stream",
          "name": "stdout",
          "text": [
            "Percent of Train Data in the dataset: 10.0\n",
            "Percent of Test Data in the dataset: 45.29\n",
            "Percent of Validation Data in the dataset: 44.71\n",
            "Accuracy (Nearest Neighbour) of Test Set using Validation Set: 0.3347420892934547\n",
            "Accuracy (Random Classifier) of Test Set using Validation Set: 0.1681837884698743\n"
          ]
        }
      ],
      "source": [
        "# Small Training Set, Large Validation Set\n",
        "traindata, trainlabel, splitdata, splitlabel = split(dataset.data, dataset.target, 10/100)\n",
        "valdata, vallabel, testdata, testlabel = split(splitdata, splitlabel, 50/100)\n",
        "\n",
        "print(f\"Percent of Train Data in the dataset: {round(len(trainlabel)*100/len(dataset.target), 2)}\")\n",
        "print(f\"Percent of Test Data in the dataset: {round(len(testlabel)*100/len(dataset.target), 2)}\")\n",
        "print(f\"Percent of Validation Data in the dataset: {round(len(vallabel)*100/len(dataset.target), 2)}\")\n",
        "\n",
        "testpred = NN(testdata, testlabel, valdata)\n",
        "testpred2 = RandomClassifier(testdata, testlabel, valdata)\n",
        "\n",
        "accu = Accuracy(vallabel, testpred)\n",
        "accu2 = Accuracy(vallabel, testpred2)\n",
        "print(f\"Accuracy (Nearest Neighbour) of Test Set using Validation Set: {accu}\")\n",
        "print(f\"Accuracy (Random Classifier) of Test Set using Validation Set: {accu2}\")\n",
        "\n"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": 17,
      "metadata": {
        "colab": {
          "base_uri": "https://localhost:8080/"
        },
        "id": "isrmNfsnB96A",
        "outputId": "3ef0c3ec-696a-45fc-bb70-5ed27654a837"
      },
      "outputs": [
        {
          "output_type": "stream",
          "name": "stdout",
          "text": [
            "Percent of Train Data in the dataset: 80.43\n",
            "Percent of Test Data in the dataset: 10.01\n",
            "Percent of Validation Data in the dataset: 9.56\n",
            "Accuracy (Nearest Neighbour) of Test Set using Validation Set: 0.30410542321338063\n",
            "Accuracy (Random Classifier) of Test Set using Validation Set: 0.16421692853522554\n"
          ]
        }
      ],
      "source": [
        "# Large Training Set, Small Validation Set\n",
        "traindata, trainlabel, splitdata, splitlabel = split(dataset.data, dataset.target, 80/100)\n",
        "valdata, vallabel, testdata, testlabel = split(splitdata, splitlabel, 50/100)\n",
        "\n",
        "print(f\"Percent of Train Data in the dataset: {round(len(trainlabel)*100/len(dataset.target), 2)}\")\n",
        "print(f\"Percent of Test Data in the dataset: {round(len(testlabel)*100/len(dataset.target), 2)}\")\n",
        "print(f\"Percent of Validation Data in the dataset: {round(len(vallabel)*100/len(dataset.target), 2)}\")\n",
        "\n",
        "testpred = NN(testdata, testlabel, valdata)\n",
        "testpred2 = RandomClassifier(testdata, testlabel, valdata)\n",
        "\n",
        "accu = Accuracy(vallabel, testpred)\n",
        "accu2 = Accuracy(vallabel, testpred2)\n",
        "print(f\"Accuracy (Nearest Neighbour) of Test Set using Validation Set: {accu}\")\n",
        "print(f\"Accuracy (Random Classifier) of Test Set using Validation Set: {accu2}\")\n",
        "\n"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": 18,
      "metadata": {
        "colab": {
          "base_uri": "https://localhost:8080/"
        },
        "id": "9DsggXB5CEuh",
        "outputId": "e08eb8cf-7d44-4881-f6dc-a5945994fdb5"
      },
      "outputs": [
        {
          "output_type": "stream",
          "name": "stdout",
          "text": [
            "Percent of Train Data in the dataset: 49.54\n",
            "Percent of Test Data in the dataset: 25.22\n",
            "Percent of Validation Data in the dataset: 25.24\n",
            "Accuracy (Nearest Neighbour) of Test Set using Validation Set: 0.3211132437619962\n",
            "Accuracy (Random Classifier) of Test Set using Validation Set: 0.16065259117082534\n"
          ]
        }
      ],
      "source": [
        "# Balanced Datasets\n",
        "traindata, trainlabel, splitdata, splitlabel = split(dataset.data, dataset.target, 50/100)\n",
        "valdata, vallabel, testdata, testlabel = split(splitdata, splitlabel, 50/100)\n",
        "\n",
        "print(f\"Percent of Train Data in the dataset: {round(len(trainlabel)*100/len(dataset.target), 2)}\")\n",
        "print(f\"Percent of Test Data in the dataset: {round(len(testlabel)*100/len(dataset.target), 2)}\")\n",
        "print(f\"Percent of Validation Data in the dataset: {round(len(vallabel)*100/len(dataset.target), 2)}\")\n",
        "\n",
        "testpred = NN(testdata, testlabel, valdata)\n",
        "testpred2 = RandomClassifier(testdata, testlabel, valdata)\n",
        "\n",
        "accu = Accuracy(vallabel, testpred)\n",
        "accu2 = Accuracy(vallabel, testpred2)\n",
        "print(f\"Accuracy (Nearest Neighbour) of Test Set using Validation Set: {accu}\")\n",
        "print(f\"Accuracy (Random Classifier) of Test Set using Validation Set: {accu2}\")\n"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "qPvG4Q9cCKul"
      },
      "source": [
        "In Train-Validation-Set Split we get highest accuracy for the 10-45-45 Split, *But in Reality* the performance of the model is decreased becuase the train set is small when compared to the test and validation set.\n",
        "\n",
        "50-25-25 Split is the best in my opinion"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "PnYvkAZLQY7h"
      },
      "source": [
        "## Multiple Splits\n",
        "\n",
        "One way to get more accurate estimates for the test accuracy is by using <b>crossvalidation</b>. Here, we will try a simple version, where we do multiple train/val splits and take the average of validation accuracies as the test accuracy estimation. Here is a function for doing this. Note that this function will take a long time to execute."
      ]
    },
    {
      "cell_type": "code",
      "execution_count": 19,
      "metadata": {
        "id": "E4nGCUQXBTzo"
      },
      "outputs": [],
      "source": [
        "# you can use this function for random classifier also\n",
        "def AverageAccuracy(alldata, alllabel, splitpercent, iterations, classifier=NN):\n",
        "  accuracy = 0\n",
        "  for ii in range(iterations):\n",
        "    traindata, trainlabel, valdata, vallabel = split(alldata, alllabel, splitpercent)\n",
        "    valpred = classifier(traindata, trainlabel, valdata)\n",
        "    accuracy += Accuracy(vallabel, valpred)\n",
        "  return accuracy/iterations # average of all accuracies"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": 20,
      "metadata": {
        "colab": {
          "base_uri": "https://localhost:8080/"
        },
        "id": "H3qtNar7Bbik",
        "outputId": "8a400ce4-dbf6-425f-9b35-0e88b57804d2"
      },
      "outputs": [
        {
          "output_type": "stream",
          "name": "stdout",
          "text": [
            "Average validation accuracy is  0.3387104262363516\n",
            "test accuracy is  0.8680115273775216\n"
          ]
        }
      ],
      "source": [
        "print('Average validation accuracy is ', AverageAccuracy(alltraindata, alltrainlabel, 75/100, 10, classifier=NN))\n",
        "testpred = NN(alltraindata, alltrainlabel, testdata)\n",
        "print('test accuracy is ',Accuracy(testlabel, testpred))"
      ]
      },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "33GIn4x5VH-d"
      },
      "source": [
        "This is a very simple way of doing cross-validation. There are many well-known algorithms for cross-validation, like k-fold cross-validation, leave-one-out etc. This will be covered in detail in a later module. For more information about cross-validation, check <a href=https://en.wikipedia.org/wiki/Cross-validation_(statistics)>Cross-validatioin (Wikipedia)</a>\n",
        "\n",
        "### Questions\n",
        "1. Does averaging the validation accuracy across multiple splits give more consistent results?\n",
        "2. Does it give more accurate estimate of test accuracy?\n",
        "3. What is the effect of the number of iterations on the estimate? Do we get a better estimate with higher iterations?\n",
        "4. Consider the results you got for the previous questions. Can we deal with a very small train dataset or validation dataset by increasing the iterations?\n"
      ]
    },
    {
      "cell_type": "markdown",
      "source": [
        "> Does averaging the validation accuracy across multiple splits give more consistent results?\n",
        "\n",
        "Yes, averaging the validation accuracy across multiple splits in a cross-validation process does give more consistent and reliable results.\n",
        "Because it reduces Variance of the data.\n",
        "\n",
        "(With a single validation split, the model's performance may be highly influenced by the specific data points included in that split, leading to variance in the performance estimate. Averaging over multiple splits smooths out this variance.)\n",
        "\n",
        "> Does it give more accurate estimate of test accuracy?\n",
        "\n",
        "Cross-validation provides a good estimate of how well your model should generalize to new, unseen data, which is valuable for model selection, hyperparameter tuning, and assessing generalization performance."
      ],
      "metadata": {
        "id": "6vclOei7J0Rb"
      }
    },
    {
      "cell_type": "code",
      "source": [
        "# Q3. What is the effect of the number of iterations on the estimate?\n",
        "# Do we get a better estimate with higher iterations?\n",
        "# Let's test with the iterations of 100\n",
        "\n",
        "print('Average validation accuracy is ', AverageAccuracy(alltraindata, alltrainlabel, 75/100, 100, classifier=NN))\n",
        "testpred = NN(alltraindata, alltrainlabel, testdata) # Not to mention, it took 4m to execute\n",
        "print('test accuracy(100 iterations) is ',Accuracy(testlabel, testpred))\n"
      ],
      "metadata": {
        "colab": {
          "base_uri": "https://localhost:8080/"
        },
        "id": "Nfly61hRKzpU",
        "outputId": "039f90ca-9147-4aba-fb4f-0bc3e53d6bad"
      },
      "execution_count": 21,
      "outputs": [
        {
          "output_type": "stream",
          "name": "stdout",
          "text": [
            "Average validation accuracy is  0.3400074582036098\n",
            "test accuracy(100 iterations) is  0.8680115273775216\n"
          ]
        }
      ]
    },
    {
      "cell_type": "code",
      "source": [
        "# testing with 5 iterations\n",
        "print('Average validation accuracy is ', AverageAccuracy(alltraindata, alltrainlabel, 75/100, 5, classifier=NN))\n",
        "testpred = NN(alltraindata, alltrainlabel, testdata)\n",
        "print('test accuracy(5 iterations) is ',Accuracy(testlabel, testpred))\n",
        "\n",
        "# Iterations are used to increase the accuracy of the model, but to some extent.\n",
        "# Increase in the iterations can lead to increase in runtime, accuracy is same in all the test cases.\n"
      ],
      "metadata": {
        "colab": {
          "base_uri": "https://localhost:8080/"
        },
        "id": "zWJ8pic5OfLl",
        "outputId": "20ed4ddf-daa1-4ba9-c139-62de3c1e4075"
      },
      "execution_count": 23,
      "outputs": [
        {
          "output_type": "stream",
          "name": "stdout",
          "text": [
            "Average validation accuracy is  0.3372611591497087\n",
            "test accuracy(5 iterations) is  0.8680115273775216\n"
          ]
        }
      ]
    },
    {
      "cell_type": "code",
      "source": [
        "# Q4.Can we deal with a very small train dataset or validation dataset by increasing the iterations?\n",
        "# Here I take 10% of the dataset as the train data\n",
        "print(\"One Iteration\\n\")\n",
        "print('Average validation accuracy is ', AverageAccuracy(alltraindata, alltrainlabel, 0.01, 1, classifier=NN))\n",
        "testpred = NN(alltraindata, alltrainlabel, testdata)\n",
        "print('test accuracy(1 iterations) is ',Accuracy(testlabel, testpred))\n",
        "print(\"50 Iterations\\n\")\n",
        "print('Average validation accuracy is ', AverageAccuracy(alltraindata, alltrainlabel, 0.01, 1, classifier=NN))\n",
        "testpred = NN(alltraindata, alltrainlabel, testdata)\n",
        "print('test accuracy(50 iterations) is ',Accuracy(testlabel, testpred))\n",
        "print(\"100 Iterations\\n\")\n",
        "print('Average validation accuracy is ', AverageAccuracy(alltraindata, alltrainlabel, 0.01, 1, classifier=NN))\n",
        "testpred = NN(alltraindata, alltrainlabel, testdata)\n",
        "print('test accuracy(100 iterations) is ',Accuracy(testlabel, testpred))\n"
      ],
      "metadata": {
        "colab": {
          "base_uri": "https://localhost:8080/"
        },
        "id": "vXSZWmHsRVKY",
        "outputId": "d99408df-6646-49c2-accb-d121c55d0e45"
      },
      "execution_count": 32,
      "outputs": [
        {
          "output_type": "stream",
          "name": "stdout",
          "text": [
            "One Iteration\n",
            "\n",
            "Average validation accuracy is  0.2715852687051903\n",
            "test accuracy(1 iterations) is  0.8680115273775216\n",
            "50 Iterations\n",
            "\n",
            "Average validation accuracy is  0.2451976018597822\n",
            "test accuracy(50 iterations) is  0.8680115273775216\n",
            "100 Iterations\n",
            "\n",
            "Average validation accuracy is  0.27943696450428396\n",
            "test accuracy(100 iterations) is  0.8680115273775216\n"
          ]
        }
      ]
    },
    {
      "cell_type": "markdown",
      "source": [
        "*Increasing the number of iterations (folds) in cross-validation can help when dealing with a very small training dataset or validation dataset to some extent, but it may not completely overcome the limitations of a small dataset*\n",
        "\n",
        "Sometimes, the accuracy fluctuates, this is because Insufficient training data can lead to models that underfit and do not learn the underlying patterns effectively.\n",
        "\n",
        "As we increase the iterations, we increases the runtime\n",
        "\n",
        "Small datasets often suffer from sampling bias, where the limited data may not be representative of the overall population. More iterations may not fully address this issue"
      ],
      "metadata": {
        "id": "vekHWlKmTgcP"
      }
    },
    {
      "cell_type": "markdown",
      "source": [
        ">"
      ],
      "metadata": {
        "id": "861SW5IrJvZk"
      }
    }
  ],
  "metadata": {
    "colab": {
      "provenance": [],
      "include_colab_link": true
    },
    "kernelspec": {
      "display_name": "Python 3",
      "name": "python3"
    },
    "language_info": {
      "name": "python"
    }
  },
  "nbformat": 4,
  "nbformat_minor": 0
}